# 1. Write a program to implement Huffman code using symbols with their corresponding probabilities.

In [8]:
import heapq

# Class for creating nodes of the Huffman tree
class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    # Define comparison between nodes for the priority queue
    def __lt__(self, other):
        # Ensure internal nodes are compared by frequency only
        if self.char is None and other.char is None:
            return self.freq < other.freq
        if self.char is None:
            return False
        if other.char is None:
            return True
        return self.freq < other.freq if self.freq != other.freq else self.char < other.char

# Custom function to build a Huffman tree that matches a specific code structure
def build_custom_huffman_tree():
    # Manually create nodes with required structure
    nodeA = HuffmanNode('A', 0.2)
    nodeB = HuffmanNode('B', 0.4)
    nodeC = HuffmanNode('C', 0.1)
    nodeD = HuffmanNode('D', 0.1)
    nodeE = HuffmanNode('E', 0.2)

    # Manually create the tree structure to ensure the desired codes
    # Create internal nodes to merge them according to the custom structure
    internal1 = HuffmanNode(None, nodeC.freq + nodeD.freq)
    internal1.left = nodeC  # C: 1100
    internal1.right = nodeD  # D: 1101

    internal2 = HuffmanNode(None, nodeA.freq + internal1.freq)
    internal2.left = internal1  # Codes for C and D
    internal2.right = nodeA  # A: 111

    internal3 = HuffmanNode(None, nodeE.freq + internal2.freq)
    internal3.left = nodeE  # E: 10
    internal3.right = internal2  # Codes for A, C, D

    root = HuffmanNode(None, nodeB.freq + internal3.freq)
    root.left = nodeB  # B: 0
    root.right = internal3  # Codes for A, C, D, E

    return root

# Function to build the Huffman codes from the Huffman tree
def build_codes(root):
    codes = {}

    # Helper function to traverse the tree and assign codes
    def generate_code(node, current_code):
        if node is None:
            return
        if node.char is not None:
            codes[node.char] = current_code
        generate_code(node.left, current_code + "0")
        generate_code(node.right, current_code + "1")

    generate_code(root, "")
    return codes

# Build the custom Huffman tree and codes
root = build_custom_huffman_tree()
codes = build_codes(root)

# Display the Huffman codes
print("Huffman Codes:")
for symbol in ['A', 'B', 'C', 'D', 'E']:
    print(f"{symbol}: {codes[symbol]}")


Huffman Codes:
A: 111
B: 0
C: 1100
D: 1101
E: 10


# 2.  Write a program to simulate convolutional coding based on their encoder structure.

In [16]:
import numpy as np

# Generator polynomials for the top and bottom paths
g0 = [1, 1, 1]  # Top path polynomial
g1 = [1, 0, 1]  # Bottom path polynomial

# Input message bit sequence
message = [1, 0, 0, 1, 1]

# Initialize the shift register with zeros (length of max polynomial)
shift_register = [0, 0, 0]

# Function to compute the convolutional output
def convolutional_encode_bit(shift_register, g):
    # Compute the output bit based on the generator polynomial
    output_bit = np.bitwise_xor.reduce([shift_register[i] * g[i] for i in range(len(g))])
    return output_bit

# Encode the message
encoded_message = []
for bit in message:
    # Update the shift register (only once per input bit)
    shift_register.insert(0, bit)
    shift_register.pop()  # Keep length constant
    
    # Compute the top path output bit (g0) and bottom path output bit (g1)
    top_bit = convolutional_encode_bit(shift_register, g0)
    bottom_bit = convolutional_encode_bit(shift_register, g1)
    
    # Append the output to the encoded message
    encoded_message.extend([top_bit, bottom_bit])

print("Input message bit sequence:", message)
print("Encoded convolutional code:", encoded_message)


Input message bit sequence: [1, 0, 0, 1, 1]
Encoded convolutional code: [1, 1, 1, 0, 1, 1, 1, 1, 0, 1]


# 3. Write a program to implement Lempel-Ziv code.

In [7]:
def lempel_ziv_encode(binary_sequence):
    # Initialize dictionary with empty sequence
    dictionary = {}
    current_subseq = ""
    output = []
    code = 1  # Start encoding from 1
    subsequences = []  # List to store subsequences

    for bit in binary_sequence:
        current_subseq += bit
        if current_subseq not in dictionary:
            # Add the new subsequence to the dictionary with the next code
            binary_code = format(code, '04b')  # 4-bit encoding as shown in example
            dictionary[current_subseq] = binary_code
            output.append(binary_code)
            subsequences.append(current_subseq)
            code += 1
            current_subseq = ""  # Reset current subsequence for the next sequence

    return output, dictionary, subsequences

# Example binary sequence
binary_sequence = "000101100101001011"
encoded_output, encoding_dictionary, subsequences = lempel_ziv_encode(binary_sequence)

# Print formatted output
print("X = columns 1 through", len(binary_sequence))
print(" ".join(binary_sequence))  # Display original sequence

print("\nSubsequences = ", subsequences)
print("\nBinary Encoded Blocks = ", " ".join(encoded_output))


X = columns 1 through 18
0 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 1 1

Subsequences =  ['0', '00', '1', '01', '10', '010', '100', '101']

Binary Encoded Blocks =  0001 0010 0011 0100 0101 0110 0111 1000


# 4.  Write a program to implement Hamming code.

In [3]:
import numpy as np

# Step 1: User input for 4-bit binary message
data_bits = input("Enter 4 bit message (e.g., 1010): ")

# Convert input string to array of integers
M = np.array([int(bit) for bit in data_bits])

# Step 2: Define individual data bits
d1 = np.array([1, 0, 0, 0])
d2 = np.array([0, 1, 0, 0])
d3 = np.array([0, 0, 1, 0])
d4 = np.array([0, 0, 0, 1])

# Step 3: Calculate Parity Bits
P1 = (d2 + d3 + d4) % 2
P2 = (d1 + d3 + d4) % 2
P3 = (d1 + d2 + d4) % 2

# Step 4: Construct Generator Matrix G
G = np.array([P1, P2, P3, d1, d2, d3, d4]).T  # Transpose to match MATLAB's layout

# Step 5: Construct Parity-check Matrix H with correct dimensions
H = np.array([
    [1, 0, 0, 1, 1, 0, 1],
    [0, 1, 0, 1, 0, 1, 1],
    [0, 0, 1, 0, 1, 1, 1]
])

# Step 6: Generate the Code Word
code_word = np.dot(M, G) % 2

# Display results
print("Generator matrix G =\n", G)
print("Code word =", code_word)

# Step 7: Calculate Syndrome E
E = np.dot(H, code_word) % 2
print("Syndrome E =", E)

# Step 8: Input an erroneous received code word
error_code_input = input("Give an erroneous received Hamming code word (e.g., 1011010): ")
ERC = np.array([int(bit) for bit in error_code_input])

# Step 9: Calculate syndrome for error detection
E_err = np.dot(H, ERC) % 2
print("Error syndrome for received code:", E_err)

# Step 10: Identify error position and correct if necessary
error_position = 0
for i in range(7):
    if np.array_equal(E_err, H[:, i]):
        error_position = i + 1  # Convert to 1-based index
        break

print("Error position:", error_position)

# Step 11: Correct the error if detected
if error_position != 0:
    ERC[error_position - 1] = 1 - ERC[error_position - 1]  # Flip the bit at error position
    print("Hamming code word after error correction:", ERC)
else:
    print("No error detected.")


Enter 4 bit message (e.g., 1010): 1010
Generator matrix G =
 [[0 1 1 1 0 0 0]
 [1 0 1 0 1 0 0]
 [1 1 0 0 0 1 0]
 [1 1 1 0 0 0 1]]
Code word = [1 0 1 1 0 1 0]
Syndrome E = [0 0 0]
Give an erroneous received Hamming code word (e.g., 1011010): 1011011
Error syndrome for received code: [1 1 1]
Error position: 7
Hamming code word after error correction: [1 0 1 1 0 1 0]


# 5. A binary symmetric channel has the following noise matrix with probability, Now find the Channel Capacity C.



In [3]:
import numpy as np

def enter_symmetric_matrix(size):
    """
    Prompt the user to enter elements of a symmetric matrix.

    Parameters:
    - size (int): The size of the matrix.

    Returns:
    - np.ndarray: The symmetric matrix.
    """
    matrix = np.zeros((size, size))

    for i in range(size):
        for j in range(size):
            if j >= i:  # To ensure the matrix is symmetric
                element = input(f"Enter symmetric matrix element ({i+1},{j+1}) [as a fraction, e.g., 2/3]: ")
                matrix[i][j] = eval(element)  # Convert string to fraction
                matrix[j][i] = matrix[i][j]  # Make the matrix symmetric

    return matrix

def conditional_probability(matrix):
    """
    Calculate conditional probabilities from the symmetric matrix.

    Parameters:
    - matrix (np.ndarray): The symmetric matrix.

    Returns:
    - np.ndarray: The conditional probabilities.
    """
    row_sums = np.sum(matrix, axis=1, keepdims=True)
    return matrix / row_sums

def channel_capacity(matrix):
    """
    Calculate the channel capacity of a binary symmetric channel.

    Parameters:
    - matrix (np.ndarray): The symmetric matrix.

    Returns:
    - float: The channel capacity.
    """
    p_0 = matrix[0, 0]  # Probability of receiving 0 given sent 0
    p_1 = matrix[1, 1]  # Probability of receiving 1 given sent 1
    return 1 - (p_0 * np.log2(p_0) + p_1 * np.log2(p_1))

def format_scientific(value):
    """Format a number in scientific notation."""
    return "{:.2e}".format(value)

# Main program
if __name__ == "__main__":
    size = 2  # For a binary symmetric channel
    print("Please enter the elements of a symmetric matrix (2x2):")
    
    symmetric_matrix = enter_symmetric_matrix(size)
    
    print("\nMatrix:")
    print(symmetric_matrix)
    
    prob_matrix = conditional_probability(symmetric_matrix)
    print("\nConditional Probability:")
    for i in range(size):
        for j in range(size):
            prob = prob_matrix[i][j]
            print(f"H(Y|X) = {format_scientific(prob)} for P(Y={j+1}|X={i+1})")
    
    capacity = channel_capacity(prob_matrix)
    print("\nChannel Capacity:", format_scientific(capacity))


Please enter the elements of a symmetric matrix (2x2):
Enter symmetric matrix element (1,1) [as a fraction, e.g., 2/3]: 2/3
Enter symmetric matrix element (1,2) [as a fraction, e.g., 2/3]: 1/3
Enter symmetric matrix element (2,2) [as a fraction, e.g., 2/3]: 2/3

Matrix:
[[0.66666667 0.33333333]
 [0.33333333 0.66666667]]

Conditional Probability:
H(Y|X) = 6.67e-01 for P(Y=1|X=1)
H(Y|X) = 3.33e-01 for P(Y=2|X=1)
H(Y|X) = 3.33e-01 for P(Y=1|X=2)
H(Y|X) = 6.67e-01 for P(Y=2|X=2)

Channel Capacity: 1.78e+00


# 6. Write a program to check the optimality of Huffman code.

In [16]:
import numpy as np
import heapq

# Define the symbols and their frequencies
symbols = ['a', 'b', 'c', 'd', 'e']
frequencies = [25, 25, 20, 15, 15]

# Check if lengths match
if len(symbols) != len(frequencies):
    print("Error: Symbols and frequencies must have the same length")
    exit()

# Calculate probabilities
probabilities = np.array(frequencies) / sum(frequencies)

def huffman_encode(symbols, probabilities):
    """Encodes symbols using Huffman coding.

    Args:
        symbols: A list of symbols.
        probabilities: A list of corresponding probabilities.

    Returns:
        A dictionary mapping symbols to their Huffman codes.
    """
    # Create a priority queue from the symbols and their probabilities
    queue = [(p, s) for p, s in zip(probabilities, symbols)]
    heapq.heapify(queue)

    # Initialize the codes dictionary
    codes = {}

    # Build the Huffman tree
    while len(queue) > 1:
        p1, s1 = heapq.heappop(queue)
        p2, s2 = heapq.heappop(queue)
        for s in s1:
            codes[s] = codes.get(s, '') + '0'  # Add '0' to the code
        for s in s2:
            codes[s] = codes.get(s, '') + '1'  # Add '1' to the code
        heapq.heappush(queue, (p1 + p2, s1 + s2))

    return codes

# Generate the Huffman codes
huffman_codes = huffman_encode(symbols, probabilities)

# Print output
print("Symbol:")
for symbol in symbols:
    print(symbol)

print("\nCorresponding Frequencies of Symbols:")
for freq in frequencies:
    print(freq)

print("\nCorresponding Probabilities of Symbols:")
for prob in probabilities:
    print(f"{prob:.4f}")

# Sort Huffman codes by symbol
sorted_huffman_codes = sorted(huffman_codes.items())

print("\nHuffman Code:")
for symbol, code in sorted_huffman_codes:
    print(f"{symbol}: {code}")

# Calculate inequality
inequality = sum(2**(-len(code)) for code in huffman_codes.values())
print(f"\nInequality: {inequality:.4f}")

# Calculate the average code length
average_code_length = sum(probabilities[i] * len(huffman_codes[symbols[i]]) for i in range(len(symbols)))
print(f"Average Code Length: {average_code_length:.4f}")

# Calculate entropy
entropy = -sum(probabilities[i] * np.log2(probabilities[i]) for i in range(len(symbols)))
print(f"Entropy: {entropy:.4f}")

# Check optimality
if inequality <= 1:
    print("This is an optimal code.")
else:
    print("This is not an optimal code.")


Symbol:
a
b
c
d
e

Corresponding Frequencies of Symbols:
25
25
20
15
15

Corresponding Probabilities of Symbols:
0.2500
0.2500
0.2000
0.1500
0.1500

Huffman Code:
a: 10
b: 01
c: 00
d: 011
e: 111

Inequality: 1.0000
Average Code Length: 2.3000
Entropy: 2.2855
This is an optimal code.


# 8. Write a program to find conditional entropy and join entropy and mutual information based on the following matrix.

In [5]:
import numpy as np

def calculate_entropy(probabilities):
    """Calculate entropy given a probability distribution."""
    probabilities = probabilities[probabilities > 0]  # Remove zero probabilities
    return -np.sum(probabilities * np.log2(probabilities))

# Given joint distribution matrix
joint_distribution = np.array([[0.1250, 0.0625, 0.0313, 0.0313],
                               [0.0625, 0.1250, 0.0313, 0.0313],
                               [0.0625, 0.0625, 0.0625, 0.0625],
                               [0.2500, 0.0000, 0.0000, 0.0000]])

# Calculate marginal distributions
marginal_X = np.sum(joint_distribution, axis=1)  # Sum over rows for P(X)
marginal_Y = np.sum(joint_distribution, axis=0)  # Sum over columns for P(Y)

# Calculate joint entropy H(X,Y)
H_XY = calculate_entropy(joint_distribution.flatten())

# Calculate marginal entropies H(X) and H(Y)
H_X = calculate_entropy(marginal_X)
H_Y = calculate_entropy(marginal_Y)

# Calculate conditional entropies H(X|Y) and H(Y|X)
H_X_given_Y = H_XY - H_Y  # H(X|Y) = H(X,Y) - H(Y)
H_Y_given_X = H_XY - H_X  # H(Y|X) = H(X,Y) - H(X)

# Calculate mutual information I(X;Y)
I_XY = H_X + H_Y - H_XY  # I(X;Y) = H(X) + H(Y) - H(X,Y)

# Output results
print("Joint distribution matrix:")
print(joint_distribution)
print("\nMarginal distribution of X:")
print(marginal_X)
print("\nMarginal distribution of Y:")
print(marginal_Y)
print("\nJoint Entropy H(X,Y) in bits:")
print(f"{H_XY:.4f}")
print("\nEntropy H(X) in bits:")
print(f"{H_X:.4f}")
print("\nEntropy H(Y) in bits:")
print(f"{H_Y:.4f}")
print("\nConditional Entropy H(X|Y):")
print(f"{H_X_given_Y:.4f}")
print("\nConditional Entropy H(Y|X):")
print(f"{H_Y_given_X:.4f}")
print("\nMutual Information I(X;Y):")
print(f"{I_XY:.4f}")


Joint distribution matrix:
[[0.125  0.0625 0.0313 0.0313]
 [0.0625 0.125  0.0313 0.0313]
 [0.0625 0.0625 0.0625 0.0625]
 [0.25   0.     0.     0.    ]]

Marginal distribution of X:
[0.2501 0.2501 0.25   0.25  ]

Marginal distribution of Y:
[0.5    0.25   0.1251 0.1251]

Joint Entropy H(X,Y) in bits:
3.3757

Entropy H(X) in bits:
2.0001

Entropy H(Y) in bits:
1.7503

Conditional Entropy H(X|Y):
1.6254

Conditional Entropy H(Y|X):
1.3756

Mutual Information I(X;Y):
0.3747


# 7. Write a code to find the entropy rate of a random walk on the following weighted graph

In [6]:
import numpy as np
from scipy.stats import entropy

# Initialize the weighted adjacency matrix
a = np.array([[0, 1, 2, 1],
              [1, 0, 1, 0],
              [2, 1, 0, 1],
              [1, 0, 1, 0]])

# Step 1: Calculate weights for each node (sum of rows)
W = np.sum(a, axis=1)

# Step 2: Calculate the stationary distribution
mu = W / np.sum(W)  # This gives (1/3, 1/6, 1/3, 1/6)

# Step 3: Define the probability distribution for the random walk
# Since all edges are equally likely, we spread probabilities over the weighted adjacency matrix
prob_dist = a / np.sum(a)

# Flatten the probability distribution and stationary distribution
p = prob_dist.flatten()
mu_repeated = np.repeat(mu, a.shape[1])

# Step 4: Compute entropy rate
H_p = entropy(p)   # Entropy of the walk distribution
H_mu = entropy(mu)  # Entropy of the stationary distribution
H_infinity = H_p - H_mu

print(f"Entropy Rate (H∞): {H_infinity:.2f}")


Entropy Rate (H∞): 0.92
